# KakaoBrunch12M
KakaoBrunch12M은 [카카오 아레나에서 공개한 데이터](https://arena.kakao.com/datasets?id=2)로 [브런치 서비스](https://brunch.co.kr) 사용자를 통해 수집한 데이터입니다.

이 예제에서는 브런치 데이터에서 ALS를 활용해 특정 글과 유사한 글을 추천하는 예제와 개인화 추천 예제 두 가지를 살펴보겠습니다.

In [2]:
import buffalo.data
from buffalo.algo.als import ALS
from buffalo.algo.options import ALSOption
from buffalo.misc import aux
from buffalo.misc import log 
from buffalo.data.mm import MatrixMarketOptions

log.set_log_level(1) # set log level 3 or higher to check more information

## 데이터 불러오기

In [62]:
# 브런치 데이터를 ./data/kakao-brunch-12m/ 아래에 위치했다고 가정하겠습니다.
data_opt = MatrixMarketOptions().get_default_option()
data_opt.input = aux.Option(
    {
        'main': 'data/kakao-brunch-12m/main',
        'iid': 'data/kakao-brunch-12m/iid', 
        'uid': 'data/kakao-brunch-12m/uid'
    }
)
data_opt


{'type': 'matrix_market',
 'input': {'main': 'data/kakao-brunch-12m/main',
  'iid': 'data/kakao-brunch-12m/iid',
  'uid': 'data/kakao-brunch-12m/uid'},
 'data': {'internal_data_type': 'matrix',
  'validation': {'name': 'sample', 'p': 0.01, 'max_samples': 500},
  'batch_mb': 1024,
  'use_cache': False,
  'tmp_dir': '/tmp/',
  'path': './mm.h5py'}}

In [70]:
import os
import shutil
# KakaoBrunch12M 데이터에는 '#' 으로 시작하는 아이템과 사용자 아이디가 있는데,
# numpy에서 이런 라인은 주석으로 인식하기 때문에 다른 문자로 치환할 필요가 있습니다.
for filename in ['main', 'uid', 'iid']:
    src = f'./data/kakao-brunch-12m/{filename}'
    dest = f'./data/kakao-brunch-12m/{filename}.tmp'
    with open(src, 'r') as fin:
        with open(dest, 'w') as fout:
            while True:
                read = fin.read(4098)
                if len(read) == 0:
                    break
                read = read.replace('#', '$')
                fout.write(read)
    shutil.move(dest, src)
data = buffalo.data.load(data_opt)
data.create()

## 유사아이템 추천

In [ ]:
# ALS 알고리즘의 기본 옵션으로 파라미터를 학습을 하겠습니다.
# 앞선 예제에 비해서는 데이터가 크기 때문에 워커 개수를 늘렸습니다.
als_opt = ALSOption().get_default_option()
als_opt.num_workers = 4
model = ALS(als_opt, data=data)
model.initialize()
model.train()
model.save('brunch.als.model')

In [6]:
# https://brunch.co.kr/@brunch/148 - 작가 인터뷰 - 브랜드 마케터, 정혜윤 by 브런치팀
model.load('brunch.als.model')
similar_items = model.most_similar('@brunch_148', 5)
for rank, (item, score) in enumerate(similar_items):
    bid, aid = item.split('_')
    print(f'{rank + 1:02d}. {score:.3f} https://brunch.co.kr/{bid}/{aid}')

01. 0.994 https://brunch.co.kr/@brunch/149
02. 0.968 https://brunch.co.kr/@brunch/147
03. 0.950 https://brunch.co.kr/@brunch/144
04. 0.944 https://brunch.co.kr/@brunch/145
05. 0.934 https://brunch.co.kr/@brunch/143


브런치팀이 쓴 글중에서 아래와 같은 글들이 유사한 결과로 나왔습니다.
- https://brunch.co.kr/@brunch/149 : 글의 완성도를 높이는 팁, 맞춤법 검사
- https://brunch.co.kr/@brunch/147 : 크리에이터스 데이'글력' 후기
- https://brunch.co.kr/@brunch/144 : 글을 읽고 쓰는 것, 이 두 가지에만 집중하세요.
- https://brunch.co.kr/@brunch/145 : 10인의 에디터와 함께 하는, 브런치북 프로젝트 #6
- https://brunch.co.kr/@brunch/143 : 크리에이터스 스튜디오 '글쓰기 클래스' 후기

## 개인화 추천 예제

In [12]:
# 사용자에 대한 개인화 추천 결과는 topk_recommendation으로 얻을 수 있습니다.
# ALS 모델을 사용하는 가장 기본적인 방식입니다. 
for rank, item in enumerate(model.topk_recommendation('$424ec49fa8423d82629c73e6d5ae9408')):
    bid, aid = item.split('_')
    print(f'{rank + 1:02d}. https://brunch.co.kr/{bid}/{aid}')

01. https://brunch.co.kr/@sweetannie/145
02. https://brunch.co.kr/@intlovesong/28
03. https://brunch.co.kr/@tenbody/1164
04. https://brunch.co.kr/@dailylife/207
05. https://brunch.co.kr/@steven/179
06. https://brunch.co.kr/@conbus/43
07. https://brunch.co.kr/@bzup/281
08. https://brunch.co.kr/@deckey1985/51
09. https://brunch.co.kr/@brunch/151
10. https://brunch.co.kr/@tenbody/1305


In [25]:
# get_weighted_feature를 응용하면 임의의 관심사를 가진 사용자에게 
# 전달할 추천 결과를 만들 수 있습니다.
personal_feat = model.get_weighted_feature({
    '@lonelyplanet_3': 1, # https://brunch.co.kr/@lonelyplanet/3
    '@tube007_66': 1  # https://brunch.co.kr/@tube007/66
})
similar_items = model.most_similar(personal_feat, 10)
for rank, (item, score) in enumerate(similar_items):
    bid, aid = item.split('_')
    print(f'{rank + 1:02d}. {score:.3f} https://brunch.co.kr/{bid}/{aid}')

01. 0.991 https://brunch.co.kr/@lonelyplanet/3
02. 0.939 https://brunch.co.kr/@tube007/66
03. 0.924 https://brunch.co.kr/@leeseonyoung/38
04. 0.909 https://brunch.co.kr/@looktothesky/8
05. 0.898 https://brunch.co.kr/@juliakimcued/95
06. 0.894 https://brunch.co.kr/@koncreate/38
07. 0.894 https://brunch.co.kr/@sound4u2005/56
08. 0.894 https://brunch.co.kr/@enormous-hat/332
09. 0.894 https://brunch.co.kr/@nakyungseol/20
10. 0.883 https://brunch.co.kr/@yehyun86/80
